# Historical Data Analysis
The goal of the data is to see if we can find trends in historical data and also to compare generated (ETAS) with recorded (USGS) data

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
import numpy as np
import datetime as dt

In [2]:
csv_file = "Formatted_ETAS_Output.csv"
etas = pd.read_csv(csv_file, sep = ',', lineterminator='\n')
etas.head()

,Date,Time,Year,X,Y,Magnitude,Z\r
0,12/31/59,0:03:09.00,1960.002196,-119.0502,33.9790,6.50,8.2474
1,1/2/60,0:08:49.00,1960.006125,-115.6222,33.0793,4.25,7.9322
2,1/2/60,0:10:31.00,1960.007305,-115.6323,33.1220,3.03,8.4015
3,1/2/60,0:10:32.00,1960.007320,-115.5851,33.0745,3.03,7.9678
4,1/2/60,0:11:07.00,1960.007720,-115.6256,33.0290,3.08,7.9737


In [3]:
csv_file = "All (1960-2023).csv"
usgs = pd.read_csv(csv_file, sep = ',', lineterminator='\n', dtype={'time':str})
usgs.head()

C:\Users\deyan\AppData\Local\Temp\ipykernel_8856\3779433482.py:2: DtypeWarning: Columns (1,2,3,4,6,7,8,9,15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  usgs = pd.read_csv(csv_file, sep = ',', lineterminator='\n', dtype={'time':str})


,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource\r
0,2023-10-12T21:41:21.480Z,36.4661674,-120.8755035,15.85,3.15,ml,60,135,0.1035,0.2,...,2023-10-13T19:19:02.194Z,"19 km WNW of New Idria, CA",earthquake,0.4,0.61,0.175,21,automatic,nc,nc\r
1,2023-10-08T21:30:23.900Z,38.8271667,-122.804,1.75,3.87,mw,108,20,0.006058,0.06,...,2023-10-14T02:06:32.597Z,"7 km NW of The Geysers, CA",earthquake,0.07,0.11,NaN,3,reviewed,nc,nc\r
2,2023-10-05T03:09:58.000Z,35.041,-117.661,0.79,3.52,ml,63,40,0.1102,0.15,...,2023-10-06T21:24:55.024Z,"5 km NNW of Boron, CA",earthquake,0.12,0.32,0.15,156,reviewed,ci,ci\r
3,2023-10-01T19:29:36.760Z,40.2915,-124.2905,9.59,3.61,mw,40,115,0.0308,0.17,...,2023-10-10T16:43:18.991Z,"4 km S of Petrolia, CA",earthquake,0.36,0.21,NaN,4,reviewed,nc,nc\r
4,2023-10-01T15:41:29.620Z,40.2951667,-124.287,9.8,4.09,mw,42,105,0.02685,0.17,...,2023-10-02T02:34:55.127Z,"3 km S of Petrolia, CA",earthquake,0.37,0.23,NaN,4,reviewed,nc,nc\r


In [4]:
#converting the Date column into datetime format
etas["Date"] = pd.to_datetime(etas["Date"], errors="coerce", format="%m/%d/%y")
etas.loc[etas["Date"].dt.year > pd.Timestamp.now().year, "Date"] -= pd.DateOffset(years=100)

etas.head()

,Date,Time,Year,X,Y,Magnitude,Z\r
0,1959-12-31,0:03:09.00,1960.002196,-119.0502,33.9790,6.50,8.2474
1,1960-01-02,0:08:49.00,1960.006125,-115.6222,33.0793,4.25,7.9322
2,1960-01-02,0:10:31.00,1960.007305,-115.6323,33.1220,3.03,8.4015
3,1960-01-02,0:10:32.00,1960.007320,-115.5851,33.0745,3.03,7.9678
4,1960-01-02,0:11:07.00,1960.007720,-115.6256,33.0290,3.08,7.9737


In [5]:
#converting the Date column into datetime format
usgs["time"] = pd.to_datetime(usgs["time"], errors="coerce").dt.strftime("%Y-%m-%d")
usgs.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource\r
0,2023-10-12,36.4661674,-120.8755035,15.85,3.15,ml,60,135,0.1035,0.2,...,2023-10-13T19:19:02.194Z,"19 km WNW of New Idria, CA",earthquake,0.4,0.61,0.175,21,automatic,nc,nc\r
1,2023-10-08,38.8271667,-122.804,1.75,3.87,mw,108,20,0.006058,0.06,...,2023-10-14T02:06:32.597Z,"7 km NW of The Geysers, CA",earthquake,0.07,0.11,NaN,3,reviewed,nc,nc\r
2,2023-10-05,35.041,-117.661,0.79,3.52,ml,63,40,0.1102,0.15,...,2023-10-06T21:24:55.024Z,"5 km NNW of Boron, CA",earthquake,0.12,0.32,0.15,156,reviewed,ci,ci\r
3,2023-10-01,40.2915,-124.2905,9.59,3.61,mw,40,115,0.0308,0.17,...,2023-10-10T16:43:18.991Z,"4 km S of Petrolia, CA",earthquake,0.36,0.21,NaN,4,reviewed,nc,nc\r
4,2023-10-01,40.2951667,-124.287,9.8,4.09,mw,42,105,0.02685,0.17,...,2023-10-02T02:34:55.127Z,"3 km S of Petrolia, CA",earthquake,0.37,0.23,NaN,4,reviewed,nc,nc\r


In [6]:
#filter the dataset by Date > 1960-01-01 and Date < 2023-01-1 
etas = etas[(etas['Date'] > pd.to_datetime('1960-01-01')) & (etas['Date'] < pd.to_datetime('2023-01-01'))]

#filter the dataset by X > -123 and X < -113 and Y > 29 and Y < 39
etas = etas[etas['X'] > -123]
etas = etas[etas['X'] < -113]
etas = etas[etas['Y'] < 39]
etas = etas[etas['Y'] > 29]

etas.head()

,Date,Time,Year,X,Y,Magnitude,Z\r
1,1960-01-02,0:08:49.00,1960.006125,-115.6222,33.0793,4.25,7.9322
2,1960-01-02,0:10:31.00,1960.007305,-115.6323,33.1220,3.03,8.4015
3,1960-01-02,0:10:32.00,1960.007320,-115.5851,33.0745,3.03,7.9678
4,1960-01-02,0:11:07.00,1960.007720,-115.6256,33.0290,3.08,7.9737
5,1960-01-02,0:11:17.00,1960.007840,-115.6050,33.0276,3.61,7.9322


In [7]:
#filter the dataset by Date > 1960-01-01 and Date < 2023-01-1 
usgs = usgs[(pd.to_datetime(usgs['time']) > pd.to_datetime('1960-01-01')) & (pd.to_datetime(usgs['time']) < pd.to_datetime('2023-01-01'))]

usgs['longitude'] = pd.to_numeric(usgs['longitude'], errors='coerce')
usgs['latitude'] = pd.to_numeric(usgs['latitude'], errors='coerce')

#filter the dataset by X > -123 and X < -113 and Y > 29 and Y < 39
usgs = usgs[usgs['longitude'] > -123]
usgs = usgs[usgs['longitude'] < -113]
usgs = usgs[usgs['latitude'] < 39]
usgs = usgs[usgs['latitude'] > 29]
usgs.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource\r
240,2022-12-31,33.397500,-116.393333,3.88,4.14,mw,132,16,0.07391,0.19,...,2023-09-22T21:50:30.029Z,"16 km N of Borrego Springs, CA",earthquake,0.1,0.38,NaN,6,reviewed,ci,ci\r
241,2022-12-31,34.355667,-116.921833,4.73,3.47,mw,121,25,0.07845,0.15,...,2023-03-07T19:00:01.040Z,"11km SSE of Lucerne Valley, CA",earthquake,0.09,0.41,NaN,4,reviewed,ci,ci\r
246,2022-12-22,37.620167,-122.025000,3.82,3.34,mw,141,16,NaN,0.16,...,2023-04-20T04:34:00.806Z,"3km N of Union City, CA",earthquake,0.1,0.17,NaN,3,reviewed,nc,nc\r
262,2022-12-17,37.918167,-122.304000,5.48,3.57,mw,170,19,0.01598,0.15,...,2023-07-27T08:15:34.318Z,"1km ENE of El Cerrito, CA",earthquake,0.1,0.17,NaN,4,reviewed,nc,nc\r
263,2022-12-13,36.604667,-121.209333,8.88,3.28,ml,67,55,0.03812,0.09,...,2023-02-18T22:04:08.040Z,"10km NW of Pinnacles, CA",earthquake,0.14,0.28,0.129,72,reviewed,nc,nc\r


In [8]:
summary_stats = etas.describe(include="all")
print(summary_stats)

                                 Date        Time          Year             X  \
count                           31547       31547  31547.000000  31547.000000   
unique                            NaN       26489           NaN           NaN   
top                               NaN  6:49:17.00           NaN           NaN   
freq                              NaN           5           NaN           NaN   
mean    1991-09-14 02:33:35.963483008         NaN   1991.704948   -117.520496   
min               1960-01-02 00:00:00         NaN   1960.006125   -122.971200   
25%               1975-10-16 00:00:00         NaN   1975.794142   -118.711750   
50%               1992-01-15 00:00:00         NaN   1992.042660   -117.191400   
75%               2007-07-21 00:00:00         NaN   2007.554579   -116.092200   
max               2022-12-31 00:00:00         NaN   2023.001815   -113.246300   
std                               NaN         NaN     18.290538      2.080386   

                   Y     Ma

In [9]:
summary_stats = usgs.describe(include="all")
print(summary_stats)

              time      latitude     longitude  depth    mag magType    nst  \
count        28052  28052.000000  28052.000000  28042  28052   28051  26558   
unique       11447           NaN           NaN  10875    357      12    260   
top     2019-07-06           NaN           NaN      6      3      ml      0   
freq           605           NaN           NaN   2151   1492   18987   5754   
mean           NaN     35.619491   -118.254409    NaN    NaN     NaN    NaN   
std            NaN      1.926341      2.120540    NaN    NaN     NaN    NaN   
min            NaN     32.176000   -122.992000    NaN    NaN     NaN    NaN   
25%            NaN     34.032000   -119.650750    NaN    NaN     NaN    NaN   
50%            NaN     35.873500   -118.058633    NaN    NaN     NaN    NaN   
75%            NaN     37.399000   -116.390167    NaN    NaN     NaN    NaN   
max            NaN     38.996000   -113.560667    NaN    NaN     NaN    NaN   

          gap     dmin    rms  ...                 

In [10]:
# Mean magnitude for each year
print(etas.groupby(etas.Date.dt.year)['Magnitude'].mean())

# Sum of magnitudes for each year
print(etas.groupby(etas.Date.dt.year)['Magnitude'].sum())


Date
1960    3.421501
1961    3.431192
1962    3.455392
1963    3.464771
1964    3.479369
          ...   
2018    3.410692
2019    3.476648
2020    3.452177
2021    3.435720
2022    3.451487
Name: Magnitude, Length: 63, dtype: float64
Date
1960    1549.94
1961    1554.33
1962    1717.33
1963    1663.09
1964    2094.58
         ...   
2018    1626.90
2019    1866.96
2020    1712.28
2021    1669.76
2022    1763.71
Name: Magnitude, Length: 63, dtype: float64


In [11]:
time = []
for i in usgs['time']:
    time.append(pd.to_datetime(i))
usgs['time'] = time

In [12]:
usgs['mag'] = pd.to_numeric(usgs['mag'], errors='coerce')

#group usgs times by year and print mean magnitude
print(usgs.groupby(usgs['time'].dt.year)['mag'].mean())

#group usgs times by year and print sum of magnitude
print(usgs.groupby(usgs['time'].dt.year)['mag'].sum())

time
1960    3.500103
1961    3.550617
1962    3.605721
1963    3.632488
1964    3.614620
          ...   
2018    3.429632
2019    3.454067
2020    3.439794
2021    3.429510
2022    3.416330
Name: mag, Length: 63, dtype: float64
time
1960     339.5100
1961     575.2000
1962     749.9900
1963     744.6600
1964     618.1000
          ...    
2018     559.0300
2019    4611.1800
2020    2999.5000
2021    1323.7907
2022     744.7600
Name: mag, Length: 63, dtype: float64


In [13]:
# Maximum magnitude for each year
print(etas.groupby(etas.Date.dt.year)['Magnitude'].max())

# Total number of earthquakes for each year
print(etas.groupby(etas.Date.dt.year)['Magnitude'].count())

Date
1960    5.22
1961    5.78
1962    6.38
1963    5.71
1964    7.65
        ... 
2018    6.36
2019    5.57
2020    6.28
2021    6.80
2022    6.46
Name: Magnitude, Length: 63, dtype: float64
Date
1960    453
1961    453
1962    497
1963    480
1964    602
       ... 
2018    477
2019    537
2020    496
2021    486
2022    511
Name: Magnitude, Length: 63, dtype: int64


In [14]:
# Maximum usgs magnitude for each year
print(usgs.groupby(usgs['time'].dt.year)['mag'].max())

# Total number of earthquakes for each year
print(usgs.groupby(usgs['time'].dt.year)['mag'].count())

time
1960    5.00
1961    5.88
1962    5.10
1963    5.40
1964    5.20
        ... 
2018    5.29
2019    7.10
2020    6.50
2021    6.00
2022    5.06
Name: mag, Length: 63, dtype: float64
time
1960      97
1961     162
1962     208
1963     205
1964     171
        ... 
2018     163
2019    1335
2020     872
2021     386
2022     218
Name: mag, Length: 63, dtype: int64


Grouping the data into 1 month chunks and then get the same feature values


In [15]:
grouped_etas = etas.groupby(etas['Date'].dt.to_period('M')).mean()
grouped_etas.head()

TypeError: agg function failed [how->mean,dtype->object]

In [ ]:
grouped_usgs = usgs.groupby(usgs['time'].dt.to_period('M')).mean()
grouped_usgs.head()

In [ ]:
# Mean magnitude for each month
mean_mag_etas = pd.DataFrame(etas.groupby(etas['Date'].dt.to_period('M')).Magnitude.mean())
print(mean_mag_etas)

# Sum of magnitudes for each month
sum_mag_etas = pd.DataFrame(etas.groupby(etas['Date'].dt.to_period('M')).Magnitude.sum())
print(sum_mag_etas)

In [ ]:
# Mean magnitude for each month
mean_mag_usgs = pd.DataFrame(usgs.groupby(usgs['time'].dt.to_period('M')).mag.mean())
print(mean_mag_usgs)

# Sum of magnitudes for each month
sum_mag_usgs = pd.DataFrame(usgs.groupby(usgs['time'].dt.to_period('M')).mag.sum())
print(sum_mag_usgs)

In [ ]:
# Maximum magnitude for each month
max_mag_etas = pd.DataFrame(etas.groupby(etas['Date'].dt.to_period('M')).Magnitude.max())
print(max_mag_etas)

# Total number of earthquakes for each month
earthquake_count_etas = pd.DataFrame(etas.groupby(etas['Date'].dt.to_period('M')).Magnitude.count())
print(earthquake_count_etas)

In [ ]:
# Maximum magnitude for each month
max_mag_usgs = pd.DataFrame(usgs.groupby(usgs['time'].dt.to_period('M')).mag.max())
print(max_mag_usgs)

# Total number of earthquakes for each month
earthquake_count_usgs = pd.DataFrame(usgs.groupby(usgs['time'].dt.to_period('M')).mag.count())
print(earthquake_count_usgs)

In [ ]:
#plotting mean monthy magnitude
mean_mag_etas.plot(kind='line', label='ETAS', figsize = (30,10), stacked = True)
plt.title('ETAS Mean Magnitude')
mean_mag_usgs.plot(kind='line', label='USGS', figsize = (30,10), stacked = True, color = 'red')
plt.title('USGS Mean Magnitude')

In [ ]:
#plotting sum of monthy magnitude
sum_mag_etas.plot(kind='line', label='ETAS', figsize = (30,10), stacked = True)
plt.title('ETAS Sum of Monthly  Magnitude')
sum_mag_usgs.plot(kind='line', label='USGS', color = 'red', figsize = (30,10), stacked = True)
plt.title('USGS Sum of Monthly  Magnitude')

In [ ]:
#plotting max monthy magnitude
max_mag_etas.plot(kind='line', label='ETAS', figsize = (30,10), stacked = True)
plt.title('ETAS Max Monthly  Magnitude')
max_mag_usgs.plot(kind='line', label='Usgs', color = 'red', figsize = (30,10), stacked = True)
plt.title('USGS Max Monthly  Magnitude')

In [ ]:
#monthly earthquake counts
earthquake_count_etas.plot(kind='line', label='ETAS', figsize = (30,10), stacked = True)
plt.title('ETAS Number Of Eathquakes Monthly')
earthquake_count_usgs.plot(kind='line', label='USGS', color = 'red',figsize = (30,10), stacked = True)
plt.title('USGS Number Of Eathquakes Monthly')

In [ ]:
mean_mag_etas = mean_mag_etas.reset_index()
mean_mag_etas["ETAS"] = mean_mag_etas["Magnitude"]
mean_mag_etas = mean_mag_etas.drop(columns = ["Magnitude"])
mean_mag_etas.head()

In [ ]:
mean_mag_usgs = mean_mag_usgs.reset_index()
mean_mag_usgs['Date'] = mean_mag_usgs['time']
mean_mag_usgs["USGS"] = mean_mag_usgs["mag"]
mean_mag_usgs = mean_mag_usgs.drop(columns = ["mag"])
mean_mag_usgs = mean_mag_usgs.drop(columns = ["time"])
mean_mag_usgs.head()

In [ ]:
merged_mean_mag = mean_mag_etas.merge(mean_mag_usgs, on='Date')
merged_mean_mag.head()

In [ ]:
merged_mean_mag['Date'] = merged_mean_mag['Date'].astype(str)
plt.figure(figsize=(30, 10))
plt.plot(merged_mean_mag['Date'], merged_mean_mag['ETAS'], label='ETAS')
plt.plot(merged_mean_mag['Date'], merged_mean_mag['USGS'], label='USGS')
plt.legend()
plt.title('ETAS vs USGS Mean Magnitude')

In [ ]:
sum_mag_etas = sum_mag_etas.reset_index()
sum_mag_etas["ETAS"] = sum_mag_etas["Magnitude"]
sum_mag_etas = sum_mag_etas.drop(columns = ["Magnitude"])

sum_mag_usgs = sum_mag_usgs.reset_index()
sum_mag_usgs['Date'] = sum_mag_usgs['time']
sum_mag_usgs["USGS"] = sum_mag_usgs["mag"]
sum_mag_usgs = sum_mag_usgs.drop(columns = ["mag"])
sum_mag_usgs = sum_mag_usgs.drop(columns = ["time"])

merged_sum_mag = sum_mag_etas.merge(sum_mag_usgs, on='Date')
merged_sum_mag.head()

In [ ]:
merged_sum_mag['Date'] = merged_sum_mag['Date'].astype(str)
plt.figure(figsize=(30, 10))
plt.plot(merged_sum_mag['Date'], merged_sum_mag['ETAS'], label='ETAS')
plt.plot(merged_sum_mag['Date'], merged_sum_mag['USGS'], label='USGS')
plt.legend()
plt.title('ETAS vs USGS Sum Of Magnitudes')

In [ ]:
max_mag_etas = max_mag_etas.reset_index()
max_mag_etas["ETAS"] = max_mag_etas["Magnitude"]
max_mag_etas = max_mag_etas.drop(columns = ["Magnitude"])

max_mag_usgs = max_mag_usgs.reset_index()
max_mag_usgs['Date'] = max_mag_usgs['time']
max_mag_usgs["USGS"] = max_mag_usgs["mag"]
max_mag_usgs = max_mag_usgs.drop(columns = ["mag"])
max_mag_usgs = max_mag_usgs.drop(columns = ["time"])

merged_max_mag = max_mag_etas.merge(max_mag_usgs, on='Date')
merged_max_mag.head()

In [ ]:
merged_max_mag['Date'] = merged_max_mag['Date'].astype(str)
plt.figure(figsize=(30, 10))
plt.plot(merged_max_mag['Date'], merged_max_mag['ETAS'], label='ETAS')
plt.plot(merged_max_mag['Date'], merged_max_mag['USGS'], label='USGS')
plt.legend()
plt.title('ETAS vs USGS Max Magnitude')

In [ ]:
earthquake_count_etas = earthquake_count_etas.reset_index()
earthquake_count_etas["ETAS"] = earthquake_count_etas["Magnitude"]
earthquake_count_etas = earthquake_count_etas.drop(columns = ["Magnitude"])

earthquake_count_usgs = earthquake_count_usgs.reset_index()
earthquake_count_usgs['Date'] = earthquake_count_usgs['time']
earthquake_count_usgs["USGS"] = earthquake_count_usgs["mag"]
earthquake_count_usgs = earthquake_count_usgs.drop(columns = ["mag"])
earthquake_count_usgs = earthquake_count_usgs.drop(columns = ["time"])

merged_earthquake_counts = earthquake_count_etas.merge(earthquake_count_usgs, on='Date')
merged_earthquake_counts.head()

In [ ]:
merged_earthquake_counts['Date'] = merged_earthquake_counts['Date'].astype(str)
plt.figure(figsize=(30, 10))
plt.plot(merged_earthquake_counts['Date'], merged_earthquake_counts['ETAS'], label='ETAS')
plt.plot(merged_earthquake_counts['Date'], merged_earthquake_counts['USGS'], label='USGS')
plt.legend()
plt.title('ETAS vs USGS Max Magnitude')

In [ ]:
# Check for months with an average below 5 (both USGS and ETAS)
etas_mask = mean_mag_etas['Magnitude'] < 5
etas_mean = mean_mag_etas[etas_mask]
std_dev_etas = mean_mag_etas[etas_mask].std()

usgs_mask = mean_mag_usgs['mag'] < 5
usgs_mean = mean_mag_usgs[usgs_mask]
std_dev_usgs = mean_mag_usgs[usgs_mask].std()

print(etas_mean)
print(usgs_mean)
print(std_dev_etas)
print(std_dev_usgs)
# Code to plot this

NameError: name 'mean_mag_etas' is not defined

In [ ]:
# Check for months with an average between 5.0-6.0 (not inclusive; both USGS and ETAS)
etas_mask = (mean_mag_etas['Magnitude'] >= 5) and (mean_mag_etas['Magnitude'] < 6)
etas_mean = mean_mag_etas[etas_mask]
std_dev_etas = mean_mag_etas[etas_mask].std()

usgs_mask = (mean_mag_usgs['mag'] >= 5) and (mean_mag_usgs['mag'] < 6)
usgs_mean = mean_mag_usgs[usgs_mask]
std_dev_usgs = mean_mag_usgs[usgs_mask].std()

print(etas_mean)
print(usgs_mean)
print(std_dev_etas)
print(std_dev_usgs)
# Code to plot this

In [ ]:
# Check for months with an average between 6.0-7.0 (inclusive; both USGS and ETAS)
etas_mask = (mean_mag_etas['Magnitude'] >= 6) and (mean_mag_etas['Magnitude'] <= 7)
etas_mean = mean_mag_etas[etas_mask]
std_dev_etas = mean_mag_etas[etas_mask].std()

usgs_mask = (mean_mag_usgs['mag'] >= 6) and (mean_mag_usgs['mag'] <= 7)
usgs_mean = mean_mag_usgs[usgs_mask]
std_dev_usgs = mean_mag_usgs[usgs_mask].std()

print(etas_mean)
print(usgs_mean)
print(std_dev_etas)
print(std_dev_usgs)
# Code to plot this

In [ ]:
# Check for months with an average above 7 (both USGS and ETAS)
etas_mask = mean_mag_etas['Magnitude'] > 7
etas_mean = mean_mag_etas[etas_mask]
std_dev_etas = mean_mag_etas[etas_mask].std()

usgs_mask = mean_mag_usgs['mag'] > 7
usgs_mean = mean_mag_usgs[usgs_mask]
std_dev_usgs = mean_mag_usgs[usgs_mask].std()

print(etas_mean)
print(usgs_mean)
print(std_dev_etas)
print(std_dev_usgs)
# Code to plot this

In [ ]:
# Difference between USGS and ETAS mean mag
mag_difference = mean_mag_usgs - mean_mag_etas
print(mag_difference)

# Find variance between ETAS and USGS data
variance_usgs = mean_mag_usgs['mag'].var()
variance_etas = mean_mag_etas['Magnitude'].var()
variance_difference = variance_usgs - variance_etas
# Code to plot this

print(varaince_usgs)
print(variance_etas)
print(variance_difference)

# Find standard deviation between ETAS and USGS data
std_dev_usgs = mean_mag_usgs['mag'].std()
std_dev_etas = mean_mag_etas['Magnitude'].std()
std_dev_difference = std_dev_usgs - std_dev_etas

print(std_dev_usgs)
print(std_dev_etas)
print(std_dev_difference)
# Code to plot this